In [1]:
vals = list(range(100))
vals_t = tuple(vals)
vals_d = dict.fromkeys(vals)

%timeit 555 in vals
%timeit 555 in vals_t
%timeit 555 in vals_d

439 ns ± 1.59 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
383 ns ± 0.971 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
37.2 ns ± 0.0651 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [2]:
import timeit
from quantiphy import Quantity

Quantity.set_prefs(prec=2)


def measure(case1, case2, globals=None):      
    delays = []
    for case in (case1, case2):
        t = timeit.Timer(case, globals=globals)
        number, _ = t.autorange()
        total_delay = t.timeit(number=number)
        delays.append(Quantity(total_delay / number, 's'))
        
    d1, d2 = delays
    print(f"Primero:  {d1}")
    print(f"Segundo:  {d2}")
    speedup = d1 / d2
    print(f"Mejora:   {speedup:.2f} X")

In [3]:
src_list = list(range(1_000_000))
src_set = set(src_list)

bad = "999999 in src_list"
good = "999999 in src_set"

measure(bad, good, globals={"src_set": src_set, "src_list": src_list})

Primero:  4.8 ms
Segundo:  31.4 ns
Mejora:   152841.03 X


In [4]:
src_list = list(range(1_000_000))

bad = "999999 in src_list"
worse = "999999 in set(src_list)"

measure(bad, worse, globals={"src_list": src_list})

Primero:  4.84 ms
Segundo:  25.6 ms
Mejora:   0.19 X


In [5]:
import itertools

src = list(range(1_000))

bad = "src[1:]"
good = "itertools.islice(src, 1, None)"

measure(bad, good, globals={"itertools": itertools, "src": src})

Primero:  1.13 us
Segundo:  118 ns
Mejora:   9.62 X


In [6]:
src = list(range(1_000_000))

def cuad(n):
    return n ** 2

bad = "list(map(cuad, src))"
good = "[x ** 2 for x in src]"
measure(bad, good, globals={"cuad": cuad, "src": src})

Primero:  221 ms
Segundo:  194 ms
Mejora:   1.14 X


In [7]:
def bad():
    try:
        1 / 0
    except ZeroDivisionError:
        pass
        
def good():
    try:
        1 / 1
    except ZeroDivisionError:
        pass

measure(bad, good)

Primero:  197 ns
Segundo:  37.3 ns
Mejora:   5.27 X


In [8]:
from collections import deque

src_list = list(range(1000))
src_deque = deque(range(1000))

bad = "src_list.insert(0, 123)"
good = "src_deque.appendleft(123)"

measure(bad, good, globals={"src_deque": src_deque, "src_list": src_list})

Primero:  17.5 us
Segundo:  34.3 ns
Mejora:   509.80 X


In [9]:
def bad():
    final = ""
    for _ in range(10000):
        final += "x"
    return final
        
def good():
    final = []
    for _ in range(10000):
        final.append("x")
    return "".join(final)

measure(bad, good)

Primero:  616 us
Segundo:  397 us
Mejora:   1.55 X


In [10]:
def bad():
    total = 0
    for n in range(10000):
        total += n
    return total
        
def good():
    return sum(range(10000))

measure(bad, good)

Primero:  339 us
Segundo:  94 us
Mejora:   3.61 X


In [11]:
import math

FACTOR = 0.555

class Foo:
    
    def __init__(self, correction):
        self.data = list(range(1_000_000))
        self.correction = correction
        
    def bad(self):
        total = 0
        for number in self.data:
            res = math.sin(number * self.correction * FACTOR)
            total += res
        return total
    
    def good(self):
        total = 0
        correction = self.correction
        sin = math.sin
        factor = FACTOR
        for number in self.data:
            res = sin(number * correction * factor)
            total += res
        return total

foo = Foo(1.234)
measure(foo.bad, foo.good)

Primero:  151 ms
Segundo:  84.8 ms
Mejora:   1.78 X



### Copyright 2020-2025 Facundo Batista y Manuel Carlevaro

Licencia CC BY-NC-SA 4.0

Para más info visitar: https://github.com/facundobatista/libro-pyciencia/

